<a href="https://colab.research.google.com/github/fredgi123/1-e-commerce-project/blob/main/3_price_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install xgboost
!pip install prophet
!pip install statsmodels
!pip install openpyxl
!pip install keras
!pip install --upgrade tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 62.6 MB/s eta 0:00:00
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.1
    Uninstalling tensorboard-2.17.1:
      Successfully uninstalled tensorboard-2.17.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.17.1
    Uninstalling tensorflow-2.17.1:
      Successfully uninstalled tensorflow-2.17.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.18.0 which is incompatible.


In [ ]:
# ----------------------------------------
# 1. Mount Google Drive
# ----------------------------------------
from google.colab import drive
drive.mount('/content/drive')
print("Google Drive successfully mounted.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive successfully mounted.


In [ ]:
# ----------------------------------------
# 2. Set Up Autoreload
# ----------------------------------------
%load_ext autoreload
%autoreload 2
print("Autoreload extension loaded and set to mode 2.")

# ----------------------------------------
# 2.1. Import Standard Libraries
# ----------------------------------------

# --- System and Utilities ---
import os
import sys
import json
import random
import itertools
import re
import yaml
import pickle
import joblib
import itertools
import yaml
import warnings
warnings.filterwarnings("ignore", category=FutureWarning, module="pandas")
from joblib import load
import importlib
from tensorflow.keras.layers import RepeatVector


from datetime import datetime
import datetime as dt

# --- Data Handling ---
import numpy as np
import pandas as pd

# --- Visualization ---
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
from PIL import Image as PILImage
from IPython.display import Image as DisplayImage, display
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import seaborn as sns
from matplotlib.gridspec import GridSpec
from matplotlib import style

# --- TensorFlow and Keras Imports ---
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Input, InputLayer, RepeatVector, TimeDistributed
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam, RMSprop, Nadam

# --- Scikit-learn ---
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error

# --- XGBoost ---
import xgboost as xgb

# --- Prophet ---
from prophet import Prophet

# ----------------------------------------
# 3. Define Paths and Add `scripts_p` to sys.path
# ----------------------------------------

# Define the path to your custom scripts
scripts_p = '/content/drive/MyDrive/1_Pricing/scripts'

# Add `scripts_p` to sys.path if not already added
if scripts_p not in sys.path:
    sys.path.append(scripts_p)
    print(f'Added Scripts Path to sys.path: {scripts_p}')
else:
    print(f'Scripts path already in sys.path: {scripts_p}')

# ----------------------------------------
# 4. Verify the Presence of `util_functions.py`
# ----------------------------------------
util_functions_path = os.path.join(scripts_p, "util_functions.py")
if not os.path.exists(util_functions_path):
    raise FileNotFoundError(f"'util_functions.py' not found in {scripts_p}")
else:
    print(f"'util_functions.py' found at {util_functions_path}")

# ----------------------------------------
# 5. Import Custom Modules Using importlib
# ----------------------------------------
import importlib.util

def load_module(module_name, module_path):
    """
    Load a module from a given file path using importlib.

    Args:
        module_name (str): The name to assign to the module.
        module_path (str): The file path to the module.

    Returns:
        module: The loaded module object, or None if failed.
    """
    spec = importlib.util.spec_from_file_location(module_name, module_path)
    if spec is None:
        print(f"Could not load spec for '{module_name}' at '{module_path}'.")
        return None
    module = importlib.util.module_from_spec(spec)
    try:
        spec.loader.exec_module(module)
        print(f"Module '{module_name}' loaded successfully.")
        return module
    except Exception as e:
        print(f"Failed to load module '{module_name}': {e}")
        return None

# Load all custom modules
util_functions = load_module("util_functions", util_functions_path)
util_functions_synthetic_data = load_module("util_functions_synthetic_data", os.path.join(scripts_p, "util_functions_synthetic_data.py"))
data_preprocessing = load_module("data_preprocessing", os.path.join(scripts_p, "data_preprocessing.py"))
training_baseline = load_module("training_baseline", os.path.join(scripts_p, "training_baseline.py"))
training_lstm = load_module("training_lstm", os.path.join(scripts_p, "training_lstm.py"))
training_xgboost = load_module("training_xgboost", os.path.join(scripts_p, "training_xgboost.py"))
training_prophet = load_module("training_prophet", os.path.join(scripts_p, "training_prophet.py"))
visualizations = load_module("visualizations", os.path.join(scripts_p, "visualizations.py"))
predict = load_module("predict", os.path.join(scripts_p, "predict.py"))

# Check if all modules are loaded
modules = [util_functions, util_functions_synthetic_data, data_preprocessing,
           training_baseline, training_lstm, training_xgboost, training_prophet, visualizations, predict]
for module in modules:
    if module is None:
        print("One or more modules failed to load. Please check the errors above.")

# ----------------------------------------
# 6. Define and Assign Directory Paths
# ----------------------------------------

# Define directory structure
directory_structure = {
    "data": {
        "synthetic_data": {},
        "raw_data": {},
        "intermediate_data": {}
    },
    "models": {
        "temp": {},
        "prophet": {},
        "lstm": {},
        "xgboost": {},
        "scalers":{}
    },
    "notebooks": {},
    "config": {},        # Added config directory
    "dict": {},
    "predictions": {},
    "results": {},
    "scripts": {},
    "env": {},
    "static": {
        "images": {},
        "videos": {}
    },
    "temp": {}
}



default_dict_p = '/content/drive/MyDrive/1_Pricing/dict'

# Use the `create_paths` function from `util_functions_synthetic_data` module
if util_functions_synthetic_data and hasattr(util_functions_synthetic_data, 'create_paths'):
    loaded_paths = util_functions_synthetic_data.create_paths(default_dict_p, directory_structure, imports=False)
    print("Paths created successfully.")
else:
    raise AttributeError("util_functions_synthetic_data module does not have 'create_paths' function or is not loaded.")

# Assign directory paths
dict_p = loaded_paths.get('dict_p')
images_p = loaded_paths.get('images_p')
raw_data_p = loaded_paths.get('raw_data_p')
results_p = loaded_paths.get('results_p')
synthetic_data_p = loaded_paths.get('synthetic_data_p')
scripts_p = loaded_paths.get('scripts_p')
predictions_p = loaded_paths.get('predictions_p')
models_p = loaded_paths.get('models_p')
static_p = loaded_paths.get('static_p')
videos_p = loaded_paths.get('videos_p')
env_p = loaded_paths.get('env_p')
notebooks_p = loaded_paths.get('notebooks_p')
intermediate_p = loaded_paths.get('intermediate_data_p')
temp_p = loaded_paths.get('temp_p')
prophet_p = loaded_paths.get('prophet_p')
lstm_p = loaded_paths.get('lstm_p')
xgboost_p = loaded_paths.get('xgboost_p')
config_p = loaded_paths.get('config_p')
scalers_p = loaded_paths.get('scalers_p')

categorie = "soap"
categorie_path = os.path.join(synthetic_data_p, categorie)
loaded_paths['categorie_p'] = categorie_path

print("Paths assigned successfully.")

# ----------------------------------------
# 7. Restore the Original Working Directory
# ----------------------------------------
original_cwd = '/content/drive/MyDrive/1_Pricing/notebooks'  # Ensure this is your intended directory
os.chdir(original_cwd)
print(f"Working directory restored to: {os.getcwd()}")

# ----------------------------------------
# 8. Utilize Imported Modules
# ----------------------------------------

# Example usage of util_functions
if util_functions:
    print("\nFunctions available in 'util_functions':")
    print(dir(util_functions))  # List available functions and attributes

    # Example: Using a function from util_functions
    if hasattr(util_functions, 'save_config'):
        print("\n'save_config' function is available.")
        # Example usage:
        # config = {'key': 'value'}
        # util_functions.save_config(config, 'config.yaml')
    else:
        print("\n'save_config' function is NOT available in 'util_functions'.")
else:
    print("util_functions module is not loaded.")
from data_preprocessing import *
from training_baseline import *
from training_xgboost import *
from training_prophet import *
from visualizations import *
from predict import *
from util_functions_training import *
from util_functions import *
from predict_xgboost import *
from pipeline_preparation import *
from training_lstm import*
from util_functions import apply_lag_transfo_local_indices, assign_product_index, feature_product_indices
print(categorie_path)
from lstm_preprocessing import (
    calculate_max_lag,
    validate_test_len,
    slice_indices,
    apply_lagged_features_df,
    generate_lstm_sequences,
    train_split,
    log_transform_and_scale,
    lstm_pipeline_with_slicing,
    execute_pipeline,
    explore_combinations
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Autoreload extension loaded and set to mode 2.
Scripts path already in sys.path: /content/drive/MyDrive/1_Pricing/scripts
'util_functions.py' found at /content/drive/MyDrive/1_Pricing/scripts/util_functions.py
Module 'util_functions' loaded successfully.
Module 'util_functions_synthetic_data' loaded successfully.
Module 'data_preprocessing' loaded successfully.
Module 'training_baseline' loaded successfully.
Module 'training_lstm' loaded successfully.
Module 'training_xgboost' loaded successfully.
Module 'training_prophet' loaded successfully.
Module 'visualizations' loaded successfully.
Module 'predict' loaded successfully.
Paths created successfully.
Paths assigned successfully.
Working directory restored to: /content/drive/MyDrive/1_Pricing/notebooks

Functions available in 'util_functions':
['MinMaxScaler', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package

ModuleNotFoundError: No module named 'lstm_preprocessing'

In [ ]:
# Utility functions already present in util_function.py
def feature_product_indices(feature_list, num_products, predictors, p_index):
    feature_indices = [feature_list.index(feature) for feature in predictors]
    return [(feature_index * num_products) + product_index
            for feature_index in feature_indices for product_index in p_index]

def assign_product_index(param, num_products):
    indices_dict = {
        "all_idx": list(range(num_products)),
        "comp_idx": list(range(num_products // 2, num_products)),
        "our_idx": list(range(num_products // 2)),
    }
    return indices_dict.get(param, ValueError(f"Invalid parameter name: {param}"))

class GeneralConfig:
    def __init__(self, config_file, directory_structure, categorie="soap"):

        # Load configuration
        self.paths = directory_structure
        self.categorie = categorie
        self.synthetic_data_path = os.path.join(directory_structure['synthetic_data_p'], categorie)
        self.categorie_path = directory_structure['categorie_p']
        self.config_p = directory_structure["config_p"]
        self.prophet_p = directory_structure["prophet_p"]
        self.scalers_p = directory_structure["scalers_p"]  # Adding config path to the class

        with open("{}/{}".format(self.config_p,config_file), "r") as file:
            self.config = yaml.safe_load(file)

        # General parameters
        self.num_products = self.config['general']['num_products']
        self.test_len = self.config['general']['test_len']
        self.prediction_steps = self.config['general']['prediction_steps']
        self.validation_split_ratio = self.config['general']['validation_split_ratio']
        self.all_features = self.config['general']['all_features']

        self.categorie = categorie

         # Initialize full data
        self.x_arr_2d  = None
        self.y_arr_2d = None
        self.full_df = None
        self.full_arr2d_deseas = None
        self.df_full_deseas = None
        self.initialize_full_data()

    def initialize_full_data(self):
        _, _, arr_2d, idx, _ = load_array(self.categorie_path, self.categorie, self.all_features, self.num_products)
        self.num_weeks = len(idx)
        self.col_names = [f"{f}_p{i}" for f in self.all_features for i in range(self.num_products)]
        self.idx = idx.copy()
        self.date_idx = pd.date_range(start = idx[0], periods = self.num_weeks, freq="W")
        self.x_arr_2d =arr_2d.copy()
        self.y_arr_2d = arr_2d.copy()
        self.full_df = pd.DataFrame(self.y_arr_2d, columns=self.col_names, index=idx)

    def load_lstm_config(self, model_name, model_config_key):
        """
        Dynamically load LSTM configuration parameters and calculate indices.

        Args:
            model_name (str): Name of the model (e.g., "m1_lstm").
            model_config_key (str): Key in the configuration file for the model (e.g., "MODEL_1_LSTM").
        """
        config_filename = f"{model_name}_config.yml"
        config_path = os.path.join(self.config_p, config_filename)

        # Load LSTM configuration
        with open(config_path, "r") as file:
            model_config = yaml.safe_load(file).get(model_config_key, {})

        # Dynamically set attributes for all parameters
        for section, params in model_config.items():
            for key, value in params.items():
                setattr(self, f"{model_name}_{key}", value)
                print(f"Set attribute {model_name}_{key} = {value}")

        # Assign and calculate indices for X and Y
        for var in ['x', 'y']:
            product_indices_attr = f"{model_name}_{var}_product_indices"
            feature_attr = f"{model_name}_{var}_predictors" if var == 'x' else f"{model_name}_y_target"
            # Assign product indices
            indices = assign_product_index(getattr(self, product_indices_attr), self.num_products)
            setattr(self, product_indices_attr, indices)
            # Calculate feature-specific indices
            setattr(self, f"{model_name}_{var}_indices",feature_product_indices(self.all_features, self.num_products, getattr(self, feature_attr), indices),)
        # calculate and assign local_lag indices (for slicing array and apply lags)
        setattr(self, f"{model_name}_local_lag_indices", apply_lag_transfo_local_indices(getattr(self, f"{model_name}_x_apply_lag"),self.all_features,
                                                                                         getattr(self, f"{model_name}_x_product_indices"),),)
        print(f"{model_name}: All indices and parameters loaded successfully.")

    def get_lstm_config(self, model_name):
        """ example     get_lstm_config(geral, "m42")  after geral = GeneralConfig( and geral.load_lstm_config(model_name="m42", model_config_key="MODEL_1_LSTM")
        Returns:
            dict: Dictionary containing LSTM-specific parameters.
        """
        return {
            "x_predictors": getattr(self, f"{model_name}_x_predictors"),
            "y_target": getattr(self, f"{model_name}_y_target"),
            "x_product_indices": getattr(self, f"{model_name}_x_product_indices"),
            "y_product_indices": getattr(self, f"{model_name}_y_product_indices"),
            "x_apply_lag": getattr(self, f"{model_name}_x_apply_lag"),
            "back_windows": getattr(self, f"{model_name}_back_windows"),
            "prediction_steps": getattr(self, f"{model_name}_prediction_steps"),
            "offset": getattr(self, f"{model_name}_offset"),
            "test_len": self.test_len,
            "validation_len": self.test_len,
        }
    def get_feature_array(self, feature):
        feature_idx = np.arange(self.num_products) + self.num_products * self.all_features.index(feature)
        return  self.x_arr_2d[:, feature_idx]

    def get_sliced_array(self, feature, product_indices):
        """Slice the feature array for specific product indices."""
        feature_idx = np.arange(self.num_products) + self.num_products * self.all_features.index(feature)
        return self.x_arr_2d[:, feature_idx][:, product_indices]

    def get_prophet_array(self):
        """Load or train Prophet models and deseasonalize demand data."""
        self.full_arr2d_deseas, self.df_full_deseas, self.baseline_w_trend, self.models_list = load_prophet_array(
            data  = self.x_arr_2d, idx = self.date_idx, category  = self.categorie, model_p = self.paths['prophet_p'], all_features  = self.all_features,
            num_products  = self.num_products, importation =  True, model_file_name =  "prophet_model.pkl")

# Initialize GeneralConfig
geral = GeneralConfig(config_file="general_config.yml", directory_structure=loaded_paths, categorie="soap")


### 1. Lstm pre_processing pipeline

In [ ]:
geral.load_lstm_config(model_name="m42", model_config_key="MODEL_1_LSTM")
x_predictors_combinations = [["price", "demand"], ["price", "mktg", "demand"]]
combination_results = execute_pipeline(geral, x_predictors_combinations, model_name="m42")

Set attribute m42_num_lstm_layers = [2, 3]
Set attribute m42_num_units = [128, 256]
Set attribute m42_dropout_rate = [0.05, 0.1, 0.2]
Set attribute m42_recurrent_dropout = [0.05, 0.1]
Set attribute m42_optimizer = ['nadam', 'adam']
Set attribute m42_learning_rate = [0.001, 0.0005]
Set attribute m42_epochs = [75, 150]
Set attribute m42_batch_size = [32, 16]
Set attribute m42_back_windows = [4, 8, 12, 16, 20]
Set attribute m42_prediction_steps = [4, 6]
Set attribute m42_log_transform = [True]
Set attribute m42_deseasonalize = [True, False]
Set attribute m42_x_feature_range = [[-1, 1]]
Set attribute m42_y_feature_range = [[0, 1]]
Set attribute m42_offset = 0
Set attribute m42_x_features_to_offset = []
Set attribute m42_y_features_to_offset = []
Set attribute m42_x_predictors = ['demand', 'price', 'mktg']
Set attribute m42_y_target = ['demand']
Set attribute m42_x_apply_lag = {'price': [0, 1, 2], 'display': [0], 'distribution': [0], 'mktg': [0, 1], 'demand': [0, 1, 2, 3, 4]}
Set attribute 

TypeError: 'int' object is not iterable

In [ ]:
def explore_combinations(combination_results):
    count = 0
    for combin_name, combin_result in combination_results.items():
        print(f"\nCombination: {combin_name}")
        for key, value in combin_result.items():
            count += 1
            if key == 'results_array':  # Specifically check for 'results_array'
                print(f"  {key}:")
                # Iterate over back windows in the results_array
                for back_window, split in value.items():
                    print(f"    Back Window: {back_window}")
                    count += 1
                    # Optionally print the shape of sequences in the split dictionary
                    for split_key, split_value in split.items():
                        print(f"      {split_key}: Shape {split_value.shape}")
            else:
                print(f"  {key}: {value}")
    return count
count = explore_combinations(combination_results)
print("total combinations", count)


Combination: m42_combination_1
  taxonomy_name: m42_config_m42_combination_1
  x_predictors: ['price', 'demand']
  log_transform: True
  deseasonalize: True
  x_feature_range: (-1, 1)
  y_feature_range: (0, 1)
  apply_lag: {'price': [0, 1, 2], 'display': [0], 'distribution': [0], 'mktg': [0, 1], 'demand': [0, 1, 2, 3, 4]}
  X_scaler_path: /content/drive/MyDrive/1_Pricing/models/scalers/X_scaler_m42_combination_1.save
  Y_scaler_path: /content/drive/MyDrive/1_Pricing/models/scalers/Y_scaler_m42_combination_1.save
  results_array:
    Back Window: 4
      X_train_seq: Shape (94, 4, 80)
      Y_train_seq: Shape (94, 4, 10)
      X_val_seq: Shape (15, 4, 80)
      Y_val_seq: Shape (15, 4, 10)
      X_test_seq: Shape (15, 4, 80)
      Y_test_seq: Shape (15, 4, 10)
    Back Window: 8
      X_train_seq: Shape (91, 8, 80)
      Y_train_seq: Shape (91, 4, 10)
      X_val_seq: Shape (15, 8, 80)
      Y_val_seq: Shape (15, 4, 10)
      X_test_seq: Shape (15, 8, 80)
      Y_test_seq: Shape (15, 4

In [ ]:
import os
from itertools import product

# --- Data Manipulation ---
import numpy as np
import pandas as pd

# --- Model Training and Metrics ---
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


# Step 1: Calculate max_lag dynamically based on x_apply_lag
def calculate_max_lag(x_apply_lag):
    """Calculate the maximum lag required for features in x_apply_lag."""
    max_lag = max([max(lags) for lags in x_apply_lag.values()])
    return max_lag


# Step 2: Validate test_len calculation
def validate_test_len(back_window, prediction_steps, offset, x_apply_lag):
    """
    Validate and calculate test_len based on the parameters and x_apply_lag.
    """
    max_lag = calculate_max_lag(x_apply_lag)
    test_len = back_window + prediction_steps + offset + max_lag
    print(f"Calculated max_lag: {max_lag}")
    print(f"Calculated test_len: {test_len}")
    return test_len


# Step 3: Slice the indices based on predictors and targets
def slice_indices(X_offset, Y_offset, x_predictors, y_target, x_product_indices, y_product_indices):
    col_names_x = [f"{feature}_p{product_idx}" for feature in x_predictors for product_idx in x_product_indices]
    col_names_y = [f"{feature}_p{product_idx}" for feature in y_target for product_idx in y_product_indices]

    sliced_X = X_offset[col_names_x]
    sliced_Y = Y_offset[col_names_y]

    print("Sliced X shape:", sliced_X.shape)
    print("Sliced Y shape:", sliced_Y.shape)

    return sliced_X, sliced_Y


# Step 4: Apply lagged features
def apply_lagged_features_df(df, x_apply_lag):
    """Generate lagged features based on x_apply_lag."""
    max_lag = max(max(lags) for lags in x_apply_lag.values())
    lagged_cols = {
        f"{col}_lag{lag}": df[col].shift(lag)
        for feature, lags in x_apply_lag.items()
        for col in df.columns if col.startswith(f"{feature}_")
        for lag in lags
    }
    lagged_df = pd.DataFrame(lagged_cols, index=df.index).iloc[max_lag:].reset_index(drop=True)
    print(f"apply_lagged_features_df outputs: lagged_df shape: {lagged_df.shape}, max_lag: {max_lag}")
    return lagged_df, max_lag


# Step 5: Generate LSTM sequences
def generate_lstm_sequences(X, Y, back_window_size, prediction_step_size, offset):
    """Generate LSTM sequences."""
    total_samples = len(X)
    assert total_samples > back_window_size + prediction_step_size, "Insufficient samples for sequence generation."

    X_sequences, Y_sequences = [], []
    for i in range(total_samples - back_window_size - prediction_step_size + 1):
        X_seq = X[i + offset: i + back_window_size + offset]
        Y_seq = Y[i + back_window_size: i + back_window_size + prediction_step_size]
        X_sequences.append(X_seq)
        Y_sequences.append(Y_seq)

    return np.array(X_sequences), np.array(Y_sequences)


# Step 6: Train/validation/test split
def train_split(back_window, X_sequences, Y_sequences, test_len, validation_len, overlap_authorized=False):
    """Split sequences into training, validation, and test sets."""
    total_samples = len(X_sequences)
    if not overlap_authorized:
        train_end_idx = total_samples - test_len - validation_len
        val_end_idx = total_samples - test_len

        return {
            "X_train_seq": X_sequences[:train_end_idx],
            "Y_train_seq": Y_sequences[:train_end_idx],
            "X_val_seq": X_sequences[train_end_idx:val_end_idx],
            "Y_val_seq": Y_sequences[train_end_idx:val_end_idx],
            "X_test_seq": X_sequences[val_end_idx:],
            "Y_test_seq": Y_sequences[val_end_idx:],
        }
    else:
        train_end_idx = int(total_samples * 0.65)
        val_end_idx = int(total_samples * 0.85)
        val_start_idx = val_end_idx - validation_len + int(0.1 * validation_len)
        test_start_idx = -validation_len + int(0.1 * validation_len)
        return {
            "X_train_seq": X_sequences[:train_end_idx],
            "Y_train_seq": Y_sequences[:train_end_idx],
            "X_val_seq": X_sequences[val_start_idx:val_end_idx],
            "Y_val_seq": Y_sequences[val_start_idx:val_end_idx],
            "X_test_seq": X_sequences[test_start_idx:],
            "Y_test_seq": Y_sequences[test_start_idx:],
        }
def log_transform_and_scale(df, log_transform, x_feature_range, y_feature_range, taxonomy_name, saving_path):
    os.makedirs(saving_path, exist_ok=True)  # Ensure saving path exists

    X_scaler = MinMaxScaler(feature_range=x_feature_range)
    Y_scaler = MinMaxScaler(feature_range=y_feature_range)

    df_x = df.copy()
    df_y = df.copy()
    cols = df.columns  # Maintain column names

    # Apply log transformation if specified
    if log_transform:
        print("Applying log transformation to predictors and targets...")
        df_x = np.log1p(df_x)
        df_y = np.log1p(df_y)

    # Fit and transform using MinMaxScaler
    df_x = pd.DataFrame(X_scaler.fit_transform(df_x), columns=cols, index=df.index)
    df_y = pd.DataFrame(Y_scaler.fit_transform(df_y), columns=cols, index=df.index)

    # Save scalers for future use
    from joblib import dump
    X_scaler_path = os.path.join(saving_path, f"X_scaler_{taxonomy_name}.save")
    Y_scaler_path = os.path.join(saving_path, f"Y_scaler_{taxonomy_name}.save")
    dump(X_scaler, X_scaler_path)
    dump(Y_scaler, Y_scaler_path)

    print(f"Scalers saved to {X_scaler_path} and {Y_scaler_path}")

    return df_x, df_y, df_x.values, df_y.values, X_scaler_path, Y_scaler_path





In [ ]:
def train_models_for_combinations(results, baseline_hyperparams, prediction_steps, model_path, taxonomy_prefix):
    updated_results = {}

    for combin_name, combin_result in results.items():
        print(f"\nTraining for {combin_name}...")

        # Construct taxonomy name
        taxonomy_name = f"{taxonomy_prefix}_{combin_name}"

        if "results_array" not in combin_result:
            raise KeyError(f"results_array missing for {combin_name}. Available keys: {list(combin_result.keys())}")

        results_array = combin_result["results_array"]
        if not results_array:
            raise ValueError(f"results_array is empty or None for {combin_name}")

        # Train the model
        m4_best_model, m4_best_params = train_lstm_prediction_model_with_preprocessed_arrays(
            results =                   results_array,
            baseline_hyperparams =      baseline_hyperparams,
            prediction_steps =          prediction_steps,
            model_path =                lstm_p,
            taxonomy_name =             taxonomy_name,
            importation =               True  # Set to False to train the model
        )

        # Store the best model and parameters in the updated results
        updated_results[combin_name] = {
            **combin_result,  # Include all existing combination details
            "best_model": m4_best_model,
            "best_params": m4_best_params
        }

        # Log the best parameters
        print(f"Best parameters for {combin_name}: {m4_best_params}")

    return updated_results

# Call the function to train models for all combinations
phase_2_results = train_models_for_combinations(
    results =                       combination_results,  # The results dictionary from the previous pipeline
    baseline_hyperparams =          lstm_hyperparams_phase2,  # Baseline hyperparameter HERE
    prediction_steps =              4,  # here should be general config in class
    model_path =                    lstm_p,  # here should be general config
    taxonomy_prefix =               "m4_demand_lstm_phase2"
    importation = True  # Taxonomy prefix
)
print(geral.m4_lstm_hyperparams_phase2)

In [ ]:
def train_lstm_prediction_model_with_preprocessed_arrays(
    results,
    baseline_hyperparams,
    prediction_steps,
    model_path="lstm_models",
    taxonomy_name="lstm_baseline",
    importation=False
):
    """
    Train or load an LSTM model using preprocessed sequences.

    Parameters:
        - results (dict): Dictionary containing sequences for each back window.
        - baseline_hyperparams (dict): Hyperparameter grid for the model.
        - prediction_steps (int): Number of prediction steps for the model.
        - model_path (str): Directory path to save or load the model.
        - taxonomy_name (str): Unique name for saving/loading the model.
        - importation (bool): If True, loads an existing model instead of training.

    Returns:
        - best_model (tf.keras.Model): Trained or loaded LSTM model.
        - best_params (dict): Dictionary of best hyperparameters and metrics.
    """
    if not model_path:
        raise ValueError("Model path is None or invalid.")
    if not taxonomy_name:
        raise ValueError("Taxonomy name is None or invalid.")

    # Ensure the model directory exists
    os.makedirs(model_path, exist_ok=True)

    # Handle importation of pre-trained model
    if importation:
        best_model_path = os.path.join(model_path, f"{taxonomy_name}_best_model.keras")
        best_params_path = os.path.join(model_path, f"{taxonomy_name}_best_params.json")

        if not os.path.exists(best_model_path):
            raise FileNotFoundError(f"Model file not found: {best_model_path}")

        # Load model and parameters
        best_model = tf.keras.models.load_model(best_model_path)
        with open(best_params_path, 'r') as f:
            best_params = json.load(f)
        print(f"Loaded model from {best_model_path}")
        print(f"Loaded parameters from {best_params_path}")
        return best_model, best_params

    # Initialize training variables
    best_model = None
    best_score = float('inf')
    best_params = {}
    all_scores = {}

    # Early stopping and learning rate scheduler
    early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1, min_lr=1e-6)

    # Iterate over back windows
    for back_window, split in results.items():
        print(f"\nTraining for Back Window: {back_window}")
        X_train_seq = split['X_train_seq']
        Y_train_seq = split['Y_train_seq']
        X_val_seq = split['X_val_seq']
        Y_val_seq = split['Y_val_seq']

        num_features = X_train_seq.shape[2]
        num_targets = Y_train_seq.shape[2]

        # Generate all hyperparameter combinations
        hyperparams_keys = list(baseline_hyperparams.keys())
        hyperparams_values = list(baseline_hyperparams.values())
        hyperparams_combinations = list(itertools.product(*hyperparams_values))

        for idx, combination in enumerate(hyperparams_combinations, 1):
            params = dict(zip(hyperparams_keys, combination))
            print(f"Training combination {idx}/{len(hyperparams_combinations)}: {params}")

            # Build and compile the LSTM model
            model = build_lstm_model(
                params,
                back_window_size=back_window,
                num_features=num_features,
                num_targets=num_targets,
                prediction_step_size=prediction_steps
            )

            # Define model checkpoint path
            checkpoint_path = os.path.join(model_path, f"{taxonomy_name}_temp_model.keras")
            checkpoint = ModelCheckpoint(filepath=checkpoint_path, save_best_only=True, monitor='val_loss', mode='min')

            # Train the model
            history = model.fit(
                X_train_seq, Y_train_seq,
                validation_data=(X_val_seq, Y_val_seq),
                epochs=params['epochs'],
                batch_size=params['batch_size'],
                callbacks=[early_stop, lr_scheduler, checkpoint],
                verbose=1
            )

            # Evaluate the model
            Y_val_pred = model.predict(X_val_seq)
            Y_val_seq_flat = Y_val_seq.reshape(-1, Y_val_seq.shape[-1])
            Y_val_pred_flat = Y_val_pred.reshape(-1, Y_val_pred.shape[-1])

            mse = mean_squared_error(Y_val_seq_flat, Y_val_pred_flat)
            rmse = np.sqrt(mse)
            mae = mean_absolute_error(Y_val_seq_flat, Y_val_pred_flat)
            r2 = r2_score(Y_val_seq_flat, Y_val_pred_flat)

            # Compute percentage-based metrics
            epsilon = 1e-10
            denominator = np.abs(Y_val_seq_flat)
            denominator = np.where(denominator < epsilon, epsilon, denominator)
            safe_mape = np.mean(np.abs((Y_val_seq_flat - Y_val_pred_flat) / denominator)) * 100
            smape = np.mean(2 * np.abs(Y_val_seq_flat - Y_val_pred_flat) / (np.abs(Y_val_seq_flat) + np.abs(Y_val_pred_flat) + epsilon)) * 100

            # Update the best model
            if mse < best_score:
                best_score = mse
                all_scores = {
                    "mse": mse,
                    "rmse": rmse,
                    "mae": mae,
                    "r2": r2,
                    "safe_mape": safe_mape,
                    "smape": smape
                }
                best_model = model
                best_params = params.copy()
                best_params.update({
                    'taxonomy': taxonomy_name,
                    'back_window': back_window,
                    'prediction_steps': prediction_steps,
                    'num_features': num_features,
                    'num_targets': num_targets,
                    'best_score': best_score,
                    'all_scores': all_scores
                })
                print(f"New best model for back_window={back_window} with MSE={mse:.4f}")

    # Save the best model and parameters
    best_model_path = os.path.join(model_path, f"{taxonomy_name}_best_model.keras")
    best_params_path = os.path.join(model_path, f"{taxonomy_name}_best_params.json")

    best_model.save(best_model_path)
    with open(best_params_path, 'w') as f:
        json.dump(best_params, f)

    print(f"\nBest model saved to {best_model_path}")
    print(f"Best parameters saved to {best_params_path}")

    return best_model, best_params

